# Selection: Assertions

This section details how to use the `assert_selections()` method with your extractions.

The purpose of this method is _not_ to alter the selection in any way, it is to let you assert (confirm) you have **selected the cell values that you were expecting**.

| <span style="color:green">Note - there is more information on Assertions vs Validation in the [best practice](/../intermediate/best-practice) section later in this documentation. Our purpose here is just to understand how the assertion wrapper is used.</span>|
|-----------------------------------------|

## Source Data

The data source we're using for these examples is shown below:

The [full data source can be viewed here](https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv).

In [1]:
from datachef import acquire, preview, CsvSelectable

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")
preview(table)

,A,B,C,D,E,F,G,H,I,J,K
1,,,,,,,,,,,
2,,,Houses,Cars,Boats,,,,Houses,Cars,Boats
3,Beatles,,,,,,Rolling Stones,,,,
4,,John,1,5,9,,,Keith,2,6,10
5,,Paul,2,6,10,,,Mick,3,7,11
6,,George,2,7,11,,,Charlie,3,8,12
7,,Ringo,4,8,12,,,Ronnie,5,9,13
8,,,,,,,,,,,


## assert_selection() : are_one_one

The `are_one_of` assertions compares the values of your selections to whatever list of strings you pass in.

So the following example will **not** raise an AssertionError:

In [2]:
from datachef import acquire, CsvSelectable

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

table.excel_ref('B').is_not_blank().assert_selections(are_one_of=["John", "Paul", "Ringo", "George"])


whereas this example **will** raise an AssertionError (as we're remove "George" from the list).

| <span style="color:green">Note - we're going to use a try catch to catch then print the exception, this is purely so it doesn't stop the execution of this notebook - you **dont** need to do anything like this in practice.</span>|
|-----------------------------------------|

In [3]:
from datachef import acquire, CsvSelectable

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

try:
    table.excel_ref('B').is_not_blank().assert_selections(are_one_of=["John", "Paul", "Ringo"])
except AssertionError as err:
    print(err)

Cell value: 'George' not in ['John', 'Paul', 'Ringo']


## assert_selection() : match

The `match` assertions compares the values of your selections to whateverregular expression you pass in.

So the following example will **not** raise an AssertionError:

In [4]:
from datachef import acquire, CsvSelectable

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

table.excel_ref('B4').assert_selections(match="John")

hereas this example **will** raise an AssertionError (as our regex is explicitly "John" but we've selected all of the Beatles).

| <span style="color:green">Note - we're going to use a try catch to catch then print the exception, this is purely so it doesn't stop the execution of this notebook - you **dont** need to do anything like this in practice.</span>|
|-----------------------------------------|

In [5]:
from datachef import acquire

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

try:
    table.excel_ref('B').is_not_blank().assert_selections(match="John")
except AssertionError as err:
    print(err)

Cell value: 'Paul' does not match regular expression 'John'


## assert_selection() : using

The `using` method allows you to pass in a function of callable python class to do your assertions.

**If you're still relatively early in your journey with python you can safely come back to this at a later time. It's useful but the standard keywords defined above are sufficant in many cases**

The callable python object passed in via the `using` keyword must:

- Be a function or a python class with a `__call__` dunder method.
- Must take one argument exactly which will be a `datachef.models.source.cell.Cell` object.
- Must raise an AssertionError if the contents of the cell are not as expected.

Please note - in keeping with our overall goals while you _can_ write these things in line, its typically better to develop your own reusable code snippets as appropriate to your own data and use cases.

Examples follow:

### using: via a function

In this instance, a function that asserts the `Cell.value` is exactly equal to "John".

In [6]:
from datachef.models.source.cell import Cell
from datachef import acquire

def assert_its_john(cell: Cell):
    """
    Assert the value of the cell is equal to exactly 'John'
    """
    assert cell.value == "John", f"Cell value is {cell.value}, expected 'John'"

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

try:
    table.excel_ref('B').is_not_blank().assert_selections(using=assert_its_john)
except AssertionError as err:
    print(err)

Cell value is Paul, expected 'John'


### using: via a class

In this instance, we're going to define a class that asserts the `Cell.value` is exactly equal to one of the names of a Beatle.

Note - you could implement this funcionality easier with `are_one_of` keyword, but it makes for a nice clear example.

In this instance we're **not** going to include Ringo, so an assertion error will be raised.

In [8]:
from typing import List

from datachef.models.source.cell import Cell
from datachef import acquire, CsvSelectable

class ValueAsserter:
    """
    Class to check that the value of a cell is
    present within the list provided during class
    instantiation.
    """

    def __init__(self, names: List[str]):
        self.names = names

    def __call__(self, cell: Cell):
        assert cell.value in self.names, (
            f""
        )

# Feed the name checker some names
name_checker = ValueAsserter(["John", "Paul", "George"])

table: CsvSelectable = acquire.csv.http("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv")

try:
    table.excel_ref('B').is_not_blank().assert_selections(using=name_checker)
except AssertionError as err:
    print(err)